In [ ]:
import spikeinterface.widgets as sw
import spikeinterface.extractors as se
import numpy as np
import os
%matplotlib notebook

# Testing original functionality

In [ ]:
# Generate test data
np.random.seed(1234)

nUnits = 8;
avgFiringRate = 10; # Hz
recordingLength = 60; # in seconds

# Generate spike times
numSpikes = np.random.poisson(avgFiringRate*nUnits);
spikeTimes = np.random.uniform(0,recordingLength,numSpikes);
spikeTimes.sort(); # .res format expects total spikes to be in order

# Generate indices of spikes from 1...nUnits + 1
spikeInds = np.random.randint(1,nUnits+1,numSpikes);

In [ ]:
print(numSpikes)
print(spikeTimes)
print(spikeInds)

In [ ]:
# Save test data
f1 = open("TestData.res","w+")
f2 = open("TestData.clu","w+")
f2.write("%d\n" % nUnits)
for j in range(numSpikes):
    f1.write("%d\n" % spikeTimes[j])
    f2.write("%d\n" % spikeInds[j])
    
f1.close() 
f2.close()

In [ ]:
testNSE = se.NeuroscopeSortingExtractor('./TestData.res',
                                        './TestData.clu')

In [ ]:
print('Units', testNSE.get_unit_ids())
for i in testNSE.get_unit_ids():
    print('Spike Times for unit', i, ': ', testNSE.get_unit_spike_train(i))
    
# Following the recent fix, this will now ignore cluster ID nUnits, since it expects it to range from 0,...,nUnits-1

In [ ]:
w_rs = sw.plot_rasters(testNSE,sampling_frequency=1) # This all works fine with the original functionality

# Testing new default functionality

In [ ]:
# Now, re-generate the data but force spikeInds to go from 0 to 7 instead of 1 to 8,
# which is the actual Neuroscope format.
#
# Additionally, the 0 index group consists of unsorted spikes
# while the 1 index group consists of multi-unit recordings as well...

# Generate test data
np.random.seed(1234)

nUnits = 8;
avgFiringRate = 10; # Hz
recordingLength = 60; # in seconds

# Generate spike times
numSpikes = np.random.poisson(avgFiringRate*nUnits);
spikeTimes = np.random.uniform(0,recordingLength,numSpikes);
spikeTimes.sort(); # .res format expects total spikes to be in order

# Generate indices of spikes
spikeInds = np.random.randint(0,nUnits,numSpikes);

In [ ]:
print(numSpikes)
print(spikeTimes)
print(spikeInds)

In [ ]:
# Save test data
f1 = open("TestData.res","w+")
f2 = open("TestData.clu","w+")
f2.write("%d\n" % nUnits)
for j in range(numSpikes):
    f1.write("%d\n" % spikeTimes[j])
    f2.write("%d\n" % spikeInds[j])
    
f1.close() 
f2.close()

In [ ]:
testNSE = se.NeuroscopeSortingExtractor('./TestData.res',
                                        './TestData.clu')

In [ ]:
print('Units', testNSE.get_unit_ids())
for i in testNSE.get_unit_ids():
    print('Spike Times for unit', i, ': ', testNSE.get_unit_spike_train(i))
    
# With the old issue, this final index will be mistakenly empty
# With the new fix, it will appropriately read IDs 1,...,nUnits-1, skipping 0

In [ ]:
w_rs = sw.plot_rasters(testNSE,sampling_frequency=1) # This all works fine

# Testing new additional option to return multi-unit activity

In [ ]:
testNSE = se.NeuroscopeSortingExtractor('./TestData.res',
                                        './TestData.clu',keep_mua_units=False)

os.remove('./TestData.res')
os.remove('./TestData.clu')

In [ ]:
print('Units', testNSE.get_unit_ids())
for i in testNSE.get_unit_ids():
    print('Spike Times for unit', i, ': ', testNSE.get_unit_spike_train(i))

With the new option, we will have only nUnits-2 total units, still starting from an index of 1

In [ ]:
w_rs = sw.plot_rasters(testNSE,sampling_frequency=1) # This all works fine

# Testing neuroscope conversion method

In [ ]:
import spikeinterface.extractors as se

res_file = 'D:/BuzsakiData/SenzaiY/YutaMouse41/YutaMouse41-150903/YutaMouse41-150903.res.1'
clu_file = 'D:/BuzsakiData/SenzaiY/YutaMouse41/YutaMouse41-150903/YutaMouse41-150903.clu.1'
nwbfile_path = 'D:/BuzsakiData/SenzaiY/YutaMouse41/YutaMouse41-150903_stub.nwb'

nse = se.NeuroscopeSortingExtractor(res_file,clu_file,keep_mua_units=False)
se.NwbSortingExtractor.write_sorting(nse,nwbfile_path)